<a href="https://colab.research.google.com/github/DHRUV6029/Retrieval-Augmented-Generation--RAG--With-Mistral7B/blob/main/Topic_Specific_Crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

#!pip install tldextract
#!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 2.7 MB/s eta 0:00:00


In [2]:
import csv
import ssl
import os
from urllib.request import Request, urlopen, URLError, urljoin
from urllib.parse import urlparse
import time
import threading
import queue
from bs4 import BeautifulSoup
from bs4.element import Comment
from pathlib import Path
import re
from datetime import datetime
from queue import PriorityQueue
from matplotlib import pyplot as plt
import tldextract
import hashlib
import requests
from google.colab import files as FILE
import pathlib
import pandas as pd

from transformers import AutoTokenizer, AutoModel
from torch.nn import functional as F

tokenizer = AutoTokenizer.from_pretrained('deepset/sentence_bert')
model = AutoModel.from_pretrained('deepset/sentence_bert')


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [10]:
#Mounting google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
###this is the configuration script setting for crawling topic specific websites
### Script Settings ###
conf = dict(
    dir = '/content/drive/My Drive/Linkcrawler/Spider/',
    imgsDir = "/content/drive/My Drive/Linkcrawler/Spider1/images/",
    base_url = ['https://finance.yahoo.com/',
                'https://www.ft.com/'],

    number_of_threads = 9,
    labels = ['finance'],
    blacklist = ['facebook','instagram','youtube']
)

start_time = int(time.time())
crawled_csv = conf['dir']+'crawler_'+"{}.csv".format(start_time)
###

In [14]:
crawled_csv

'/content/drive/My Drive/Linkcrawler/Spider/crawler_1701207802.csv'

In [15]:
speed_samples_10S = [] # number of total download pages
speed_samples_10S_ctr = 0 # counter oh how many 10s sample we have
pages_count = 0

lck = threading.Lock()

#Delete the Crawler Directory to start from a clean dirctory
if os.path.exists(conf['dir']) is False:
  print("Create Directory")
  os.makedirs(conf['dir'])

if os.path.exists(conf['imgsDir']) is False:
  print("Create Imgs Directory")
  os.makedirs(conf['imgsDir'])

crawler_csv_file = open(crawled_csv, mode='a')
crawlerFile_writer = csv.writer(crawler_csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
crawlerFile_writer.writerow(["timestamp","hash","link","post_text","downloaded_imgs","processed","label","true_label"])
pages_count = pages_count +1
crawler_csv_file.close()

Create Directory
Create Imgs Directory


In [16]:
#Crawler will be implemented based on Prririty queue
class MyPriorityQueue(PriorityQueue):
    def __init__(self):
        PriorityQueue.__init__(self)
        self.counter = 0

    def put(self, item, priority):
        PriorityQueue.put(self, (priority, self.counter, item))
        self.counter += 1

    def get(self, *args, **kwargs):
        _, _, item = PriorityQueue.get(self, *args, **kwargs)
        return item

In [17]:
class Classifier(threading.Thread):
  def __init__(self):
    threading.Thread.__init__(self)

  def get_label(self,sentence):
    # run inputs through model and mean-pool over the sequence
    # dimension to get sequence-level representations
    inputs = tokenizer.batch_encode_plus([sentence] + conf['labels'],
                                        return_tensors='pt',
                                        pad_to_max_length=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    output = model(input_ids, attention_mask=attention_mask)[0]
    sentence_rep = output[:1].mean(dim=1)
    label_reps = output[1:].mean(dim=1)

    # now find the labels with the highest cosine similarities to
    # the sentence
    similarities = F.cosine_similarity(sentence_rep, label_reps)
    closest = similarities.argsort(descending=True)
    # labels = []
    label =f"{conf['labels'][closest[0]]}:{similarities[closest[0]]},{conf['labels'][closest[1]]}:{similarities[closest[1]]}"
    # for ind in closest:
      # labels.append(f"{similarities[ind]}")
    return label
    # for ind in closest:
    #     print(f'label: {labels[ind]} \t similarity: {similarities[ind]}')


def run(self):
    time.sleep(15) # sleep 10 s until the downloader get some data
    while True:
      # reading the csv file
      lck.acquire()
      df = pd.read_csv(crawled_csv,index_col=False)
      for i in range(0,len(df. index)):
        if df.loc[i, 'processed'] == "No":
          df.loc[i, 'processed'] = 'Yes'

          sentence = df.loc[i, 'post_text']
          bert_sentence = ""
          splited_sentence = sentence.split(' ')
          if len(splited_sentence) > 200 :
            for word in splited_sentence[0:200]:
              bert_sentence = bert_sentence + ' ' + word
            print(f"cut the sentence from {len(splited_sentence)} to {len(bert_sentence.split())}")
          else:
            bert_sentence = sentence

          labels = self.get_label(bert_sentence)

          df.loc[i, 'label'] = [labels]

          # new_csv = conf['dir']+'crawler_'+"{}_new_{}.csv".format(start_time,i)
          with open(crawled_csv, 'w') as f:
            df.to_csv(f,index=False)
          lck.release()
          i = i +1
          break
        i = i +1
        if i == len(df. index) - 1:
          time.sleep(10) # sleep 10 sec when there are no more enough data
      time.sleep(0.5)

In [18]:
#Crwaler Class
regular_express = re.compile(r"https?://(\.)?")

class Crawler(threading.Thread):
    def __init__(self,base_url,url_queue,have_visited, error_links,url_lock,thread_num):

        threading.Thread.__init__(self)
        print(f"Web Crawler worker {threading.current_thread()} has Started")
        self.base_url = base_url
        self.links_queue= url_queue
        self.have_visited = have_visited
        self.error_links = error_links
        self.url_lock = url_lock
        self.threadNo = thread_num

    def tag_visible(self,element): # to exclude the following tags content
        if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
            return False
        if isinstance(element, Comment):
            return False
        if re.match(r"[\n]+",str(element)): return False
        return True

    def run(self):
        global lck
        global pages_count
        my_ssl = ssl.create_default_context()
        my_ssl.check_hostname = False
        my_ssl.verify_mode = ssl.CERT_NONE

        while True:
            now = datetime.now()
            timestamp = datetime.timestamp(now)
            #print("Queue Size {} in thread{}:".format(self.links_queue.qsize(),self.threadNo))
            link = self.links_queue.get()
            link = urljoin(self.base_url,link)
            domain = tldextract.extract(link).domain
            if domain in conf['blacklist']:
              continue

            domain_have_visited = self.have_visited.get(domain,0)
            if domain_have_visited == 0:
              domain_have_visited = []

            # if the link is None the queue is exhausted or the threads are yet
            # process the links.
            if link is None:
                break

            # if The link is already visited we break the execution unless it passed 24h since last visit.
            if len(domain_have_visited) != 0:
              find=[item for item in domain_have_visited if link in item]
              if len(find)>0:
                find=tuple(find)
                if (now - find[0][1]).total_seconds() < 86400: # 86400 is 24H in secs, if the last visit time is less than 24h, then don't visit
                  # print(f"The link {link} is already visited")
                  continue
                else:
                    self.have_visited[domain].remove(find[0]) #remove the record, to add new one later

            try:
                req = Request(link, headers= {'User-Agent': 'Mozilla/5.0'})
                try:
                  response = urlopen(req, context=my_ssl)
                except:
                  continue

                soup = BeautifulSoup(response.read(),"html.parser")
                text=soup.prettify()

                images = soup.findAll('img')
                # print(f"Total {len(images)} Image Found!")

                downloaded_imgs = []
                hash = int(hashlib.sha256(link.encode('utf-8')).hexdigest(), 16) % 10**8

                if len(images) != 0:
                  for i, image in enumerate(images):
                    try:
                        # In image tag ,searching for "data-srcset"
                        image_link = image["data-srcset"]

                    # then we will search for "data-src" in img
                    # tag and so on..
                    except:
                        try:
                            # In image tag ,searching for "data-src"
                            image_link = image["data-src"]
                        except:
                            try:
                                # In image tag ,searching for "data-fallback-src"
                                image_link = image["data-fallback-src"]
                            except:
                                try:
                                    # In image tag ,searching for "src"
                                    image_link = image["src"]
                                 # if no Source URL found
                                except:
                                    pass

                    try:
                      r = requests.get(image_link).content
                      # After checking above condition, Image Download start
                      extention = pathlib.Path(image_link).suffix
                      if extention == ".png" or extention == "jpg" :
                        with open(f"{conf['imgsDir']}{hash}_{i}.{extention}", "wb") as f:
                            downloaded_imgs.append(f"{hash}_{i}")
                            # print("img saved!")
                            f.write(r)
                            f.close()
                      else:
                        pass
                    except:
                      pass
                texts = soup.findAll(text=True)

                visible_texts = filter(self.tag_visible, texts)
                text = u",".join(t.strip() for t in visible_texts)
                text = text.lstrip().rstrip()
                text = text.split(',')
                post_text = ''
                for sen in text:
                  if sen:
                    if len(sen.split(' ')) > 5: # Accept only texts with more than 5 words
                      sen = sen.rstrip().lstrip()
                      post_text += sen+' '
                if len(post_text.split(' ')) < 6:
                  continue
                lck.acquire()
                crawler_csv_file = open(crawled_csv, mode='a')
                crawlerFile_writer = csv.writer(crawler_csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                crawlerFile_writer.writerow([timestamp,hash,link,post_text,downloaded_imgs,"No","N/A"," "])
                pages_count = pages_count +1
                crawler_csv_file.close()
                lck.release()

                if type(soup.find_all('a')) == 'NoneType':
                  print("No links found in this page!")
                  continue
                else:
                  for a_tag in soup.find_all('a'):
                      links = a_tag.get("href")
                      if len(domain_have_visited) == 0:
                          self.links_queue.put(links,1)
                      else:
                          if (links not in domain_have_visited) :
                            # if (links.find('.html#') == -1): # do not crawl the intra document links
                            self.links_queue.put(links,1)

                # print(f"Adding {link} to the crawled list at {timestamp}")
                if len(domain_have_visited) != 0:
                  self.have_visited[domain].append((link,now))
                else:
                  self.have_visited[domain] = [(link,now)]
            except URLError as e:
                #print(f"URL {link} threw this error {e.reason} while trying to parse")
                self.error_links.append(link)
            finally:
                self.links_queue.task_done()